Trains and evaluate a simple MLP on the Reuters newswire topic classification task.

Adapted from: https://github.com/fchollet/keras/blob/master/examples/reuters_mlp.py

In [23]:
reset -fs

In [24]:
from keras.datasets import reuters

In [25]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1_000,
                                                         test_split=0.2)
print('Data loaded.')

print(f'{len(x_train):,} train sequences')
print(f'{len(x_test):,} test sequences')

Loading data...
Data loaded.
8,982 train sequences
2,246 test sequences


In [26]:
num_classes = max(y_train) + 1
print(num_classes, 'classes')

46 classes


In [27]:
from keras.preprocessing.text import Tokenizer

In [28]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=1_000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('Data vectorized.')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
Data vectorized.
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [31]:
from keras import utils

In [32]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [33]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [36]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(1_000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print('Model built')

Building model...
Model built


In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=True,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 1s - loss: 0.2665 - acc: 0.9329 - val_loss: 0.9235 - val_acc: 0.8053
Epoch 2/5
8083/8083 [==============================] - 1s - loss: 0.2388 - acc: 0.9365 - val_loss: 0.9373 - val_acc: 0.7976
Epoch 3/5
8083/8083 [==============================] - 1s - loss: 0.2138 - acc: 0.9414 - val_loss: 0.9744 - val_acc: 0.8042
Epoch 4/5
8083/8083 [==============================] - 1s - loss: 0.1970 - acc: 0.9475 - val_loss: 0.9947 - val_acc: 0.7931
Epoch 5/5
8083/8083 [==============================] - 1s - loss: 0.1862 - acc: 0.9490 - val_loss: 1.0026 - val_acc: 0.7931


In [43]:
score = model.evaluate(x_test, 
                       y_test,
                       batch_size=32, 
                       verbose=True)
print('Test score:', score[0])
print('Test accuracy:', score[1])

1216/2246 [===============>..............] - ETA: 0sTest score: 1.03246335843
Test accuracy: 0.789403383793
